In [ ]:
# This notebook is to be run only on Google Colaboratory primarily because of the GPU requirements

# Verify if the Nvidia GPU is enabled
!nvidia-smi

In [ ]:
# Mount the Google drive to the Google colab
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Clone the Darknet's GitHub repository
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Change the directory to the new cloned repository
%cd darknet

# Enable the OPENCV, GPU and CUDNN in the Makefile
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

In [ ]:
# Configure the training configuration file
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Make these modifications in the training configuration file

!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg

# The value of max_batches should be two thousand times the number of classes
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg

!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg

# The value of filters is 24 as there are three classes to be detected
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [ ]:
# Create the "obj.names" file in the "data" folder
!echo -e 'Green light\nRed light\nYellow light' > data/obj.names

# Create the "obj.data" file in the "data" folder
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [ ]:
# Save the yolov3_training.cfg in Google drive
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg

# Save the obj.names in Google drive
!cp data/obj.names /mydrive/yolov3/classes.txt

In [ ]:
# Make a directory
!mkdir data/obj

# Unzip the training dataset from Google drive
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:
# Write all the paths of the training images to the "train.txt" file
import glob

allTrainingImages = glob.glob("data/obj/*.jpg")

with open("data/train.txt", "w") as f:
    f.write("\n".join(allTrainingImages))

In [ ]:
# Get the darknet model 53's pre-trained weights for training
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
# Train the object detection model. This usually takes several hours.
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show